### Import spectrogram:

In [5]:
import numpy as np


Z = np.loadtxt("spectrogram_data/out.txt").T
x = np.loadtxt("spectrogram_data/xaxis.txt")
y = np.loadtxt("spectrogram_data/yaxis.txt")

print np.shape(x), np.shape(y)




(123,) (1640,)


### Calculate background

Summing over Z along x (time) and calculate ratio  Z/Backround

In [20]:
background = np.mean(Z, axis = 1)
Background = np.zeros(np.shape(Z))

""" Smoothin gbackround average over hanning window
    half window width ww"""

ww = 30

for i in range(0, np.shape(background)[0]):
    
    if i <= ww:
        background[i] = np.average(background[0:2*i+1], weights=np.hanning(2*i+1))
        
    elif i >= np.shape(background)[0]-ww:

        background[i] = np.average(background[-np.shape(background)[0]+2*i:-1], weights = np.hanning((np.shape(background)[0]-i)*2-1))
        
    else:
        background[i] = np.average(background[i-ww:i+ww], weights = np.hanning(2*ww))


for i in range(np.shape(x)[0]):
    Background[:,i] = background

rate = np.copy(Z)/Background



### plot spectrogram 

plot funtion:

In [26]:
# %load plot_spectrogram.py
"""
Created on Thu Jan 28 11:28:35 2016

@author: claessen
"""

import matplotlib.pyplot as plt

def plot_spectrogram(X, Y, Z, figno, clabel):
    
    plt.figure(figno)
    plt.contourf(X, Y, Z, 20)
    cbar = plt.colorbar()
    cbar.ax.set_ylabel(clabel)

    plt.title("spectrogam")
    plt.xlabel("time /s")
    plt.ylabel("frequency /Hz")
    plt.show()
    
    return

plot data unfiltered and filtered

In [32]:
X, Y = np.meshgrid(x, y)

plot_spectrogram(X, Y, Z*10**9, 1, "Power /nWatt" )
plot_spectrogram(X, Y, rate, 2, "rate" )

#plt.figure(3)
#plt.plot(y, backround)
#plt.show()
plt.figure(2)
plt.plot(y, Background[:,0])
plt.show()

## Furses algorithm

give score to datapoints

In [33]:
""" parameters"""

aplus = 1.0
aminus = -3.0
bplus = 1.5
bminus = 1.0
t = 1.5

scores = np.zeros(np.shape(rate))

scores[np.where(rate>=t)] = aplus*(rate[np.where(rate>=t)]-t)**bplus
scores[np.where(rate<t)] = aminus*(t-rate[np.where(rate<t)])**bminus


plot_spectrogram(X, Y, scores, 3, "scores")


-44239


#### line object

In [34]:
class line:
    def __init__(self, counter, startpos, startfreq, startscore):
        
        self.number = counter
        
        "line porperties"
        self.slope = 0
        self.length = 1
        self.score = startscore
        self.startpos = startpos
        self.startfreq = startfreq
        
        "point lists"
        self.scorelist = []
        self.binlist= []
        self.freqlist = []
        
        self.scorelist.append(self.score)
        self.binlist.append(self.startpos)
        self.freqlist.append(startfreq)

        
        
    def investigate_point(self, pos, freqin, curslice):
        
        match = 0
       
        if  np.abs(freqin-(self.startfreq+self.slope*(pos-self.startpos)))< freq_delta: #np.abs(pos-self.curpos)<=bin_delta and
            match = 1
            
            "check neighbour points"
            freqin, score = converge_function(freqin, curslice)
            
      
            "calculate new line properties"
            self.length = pos - self.startpos
            self.slope =np.mean(np.abs((freqin-np.array(self.freqlist))/(pos-np.array(self.binlist))))

            
            "collect points"
            self.freqlist.append(freqin)
            self.binlist.append(pos)
            self.scorelist.append(score)
            
            "calculate new line score"
            if self.length < lan:
                self.score = np.sum(self.scorelist)
            else:
                self.score = np.sum(self.scorelist[int(-lan):-1])
                                
            if self.score < line_thresh:
                match = 0

           
     
        return match, freq
    
    "trim edges"
    def trimming(self):

        delmyself = 0
        
        while np.array(self.scorelist[-1]) < mu*np.mean(scores[:,int(self.binlist[-1])]):
            if len(self.binlist)<2:
                break
            else:
                del self.scorelist[-1]
                del self.binlist[-1]
                del self.freqlist[-1]
                                                   
        while self.scorelist[0] < mu*np.mean(scores[:,self.binlist[0]]):
            if len(self.binlist)<2:
                break
            else:
                del self.scorelist[0]
                del self.binlist[0]
                del self.freqlist[0]  
            
        "final properties"
        self.length = len(self.binlist)
        self.startfreq = self.freqlist[0]
        self.slope = self.freqlist[-1]-self.freqlist[0]
        
        for i in range(self.length):

            j = np.where(np.abs(self.freqlist[i]-y)==np.min(np.abs(self.freqlist[i]-y)))[0][0]
            self.scorelist[i] = np.sum(scores[j-nu:j+nu,self.binlist[i]])

        return delmyself

function to inspect nearby datapoint and recalculated weighed frequency

In [35]:
def converge_function(f, s1):
    delta = freq_delta + df
    loop_counter = 0

    
    while np.abs(delta) > freq_delta:
        loop_counter += 1
        if loop_counter > 10:
            print "loop error", delta, jj
            break
        
        else:

            j = np.where(np.abs(y-f)==np.min(np.abs(y-f)))[0][0]
            score = s1[j]
            if j>bin_delta and j < np.shape(y)[0]-bin_delta:
                new_freq = np.average(y[j-bin_delta:j+bin_delta+1], weights=s1[j-bin_delta:j+bin_delta+1])

                score = np.sum(s1[j-bin_delta:j+bin_delta+1])
                delta = np.abs(new_freq-f)
                f = new_freq
                    
    f1 = f
    global slice1
    slice1[j-bin_delta:j+bin_delta+1] = -sigma

    
    return f1, score

find lines in instreaming data slices

In [39]:
" parameters" 
mu = 0.5
nu = 2
thresh = 0.05*np.max(scores)
sigma = -5.0
lan = 20.0
line_thresh = lan*thresh
print thresh



" acceptance"
bin_delta = 10
df = (y[1]-y[0])

freq_delta = df*bin_delta


" dictionary of lines"
candidates = {}
counter = 0


" instreaming slices"
slice_number = np.shape(x)[0]
s = np.copy(scores)


for ii in range(slice_number):

    freq = 0
    slice1 = s[:, ii]
    freq_list = np.where(slice1>=thresh)
    
    if len(slice1[freq_list]) > 0:


        for ll in range(len(slice1[freq_list])):  

            old_freq = freq
            freq = y[freq_list[0][ll]]

                        
            if np.abs(old_freq-freq) > freq_delta:
                match = 0 

                "let existing lines check if new point is a match"
                for jj in range(len(candidates)):
                    match, new_freq = candidates[str(jj)].investigate_point(ii, freq, slice1) 
                    
                    if match == 1:
                        freq = new_freq
                        break
                        
                "otherwise create new line"
                if match == 0:
                    freq, score = converge_function(freq, slice1)
                    candidates[str(counter)] = line(counter, ii, freq, score)

                    counter += 1
    
                    
                
print "total number of candidates: ", counter                            

candidates = {k:v for (k,v) in candidates.iteritems() if (v.slope != 0 and v.length >= lan)}
    
for mm in candidates.keys():

    candidates[mm].trimming()
    
candidates = {k:v for (k,v) in candidates.iteritems() if (v.slope != 0 and v.length >= lan)}

for mm in candidates.keys():
    print "\ncandidate ", mm
    print "start [s]: " , x[candidates[mm].startpos]
    print "frequency [MHz]: ", candidates[mm].startfreq/10**6
    print "total score: ", candidates[mm].score
    print "slope ",  candidates[mm].slope
    print "length [bins]", candidates[mm].length
#    print candidates[mm].curfreq

41.3683489616
total number of candidates:  1

candidate  0
start [s]:  0.00107368918033
frequency [MHz]:  78.3919966552
total score:  9124.60520547
slope  1374411.86257
length [bins] 38


In [90]:
line1 = candidates[str(0)]

plt.figure(1)
plt.scatter(x[line1.binlist], line1.freqlist, c=line1.scorelist)
plt.xlim([0, 0.01])
plt.ylim([np.min(y), np.max(y)])
cbar = plt.colorbar()
cbar.ax.set_ylabel('score')
plt.show()


In [78]:
print df/10**6

0.0122144791031


In [49]:
np.shape(scores)

(1640, 123)